In [1]:
import sys
import os
import numpy as np
import pandas as pd
src_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if src_dir not in sys.path:
    sys.path.append(src_dir)
from modules.preprocess import *
import json
from pymilvus.model.hybrid import BGEM3EmbeddingFunction
from pymilvus import (
    MilvusClient,
    DataType,
    Function,
    FunctionType,
    AnnSearchRequest,
    RRFRanker,
)

/workspace/Large_languige_model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
model_df=pd.read_csv("../../data/processed_tracker_info.csv")
model_df.fillna("", inplace=True)

In [10]:
model_df.columns

Index(['MODEL_ID', 'Category_Name', 'MAKER_ID', 'MAKER_NAME', 'MODEL_NAME',
       'Process_NM', 'Configuration', 'SERIAL_NO', 'VINTAGE',
       'Tracker_availability', 'Seller', 'Owner', 'P&C', 'STD_MODEL_YN',
       'STD_MAKER_YN'],
      dtype='object')

In [11]:
#장비카테고리만 진행
Equipment_list=['Asher', 'CMP', 'CVD', 'ECD', 'Etch', 'Furnace', 'Implant', 'Metrology','PVD', 'RTP', 'Stepper', 
                    'Scanner', 'Track', 'WET', 'MoCVD', 'Fab Others', 'Prober','Handler','Tester','ATE ETC','Packaging',
                    'Dicing Saw','Wire Bonder','Die Bonder','Back Grinder','PKG ETC','Chip Mounter','Reflow&Soldering',
                    'Inspection','Auto Inserter','SMT ETC']
# 원하는 장비 카테고리만 남기기
model_df = model_df[model_df['Category_Name'].isin(Equipment_list)]



In [12]:
model_df

,MODEL_ID,Category_Name,MAKER_ID,MAKER_NAME,MODEL_NAME,Process_NM,Configuration,SERIAL_NO,VINTAGE,Tracker_availability,Seller,Owner,P&C,STD_MODEL_YN,STD_MAKER_YN
0,M000,Fab Others,MA000,Unknown,Unknown,", 에틸렌한옥사이드가스소독기, AP2_TFT_STK_05_STK, Parts_A, ...",", 설비 사양: Grower 에서 성장한, DEN-ON BGA Rework St, ...",", 127DN56420, 81, ITR15624715, PNK09002EFEM, S...",", 1979, 1987, 2009, 1989, 1993, 1997, 1905, 20...","UNSPEC, RUNNING, AVAIL, NOT_AVAIL",", SUN ELECTRONIDS CO.,LTD.(SEC) , Jiangyin Cha...",", Jiangyin Changdian Advanced Packaging Co.,Lt...",", N, Y",Y,Y
1,MO1,Die Bonder,MA644,ESEC,2008 HS3,"Automatic placement , Die Bonding System , Die...",", weight : 900kg",", 207587, 207576, 207586, 207575, 207574",,"UNSPEC, AVAIL, NOT_AVAIL","Macquarie Electronics USA Inc., Dynamic Mos Te...",", Dynamic Mos Technology Co.,Ltd., JCET Group ...",N,Y,Y
3,MO1000,Stepper,MA321,Canon,FPA-1550M4W,", STEPPER, Stepper, Wafer Steppers, Stepper / ...",", , A/E, Remark : 5%?, Tool Condition : Wi...",", 912125M4, 006165M4, 11203, 005155M4, 7166, 2...",", 1992, 1995, 1991, 1993, 2000, 1999","UNSPEC, AVAIL, NOT_AVAIL",", Texas Instruments Incorporated, BOT Lease Co...",", Texas Instruments Incorporated, BOT Lease Co...",", N",Y,Y
4,MO10003,Stepper,MA2052,Seiko Instrument Inc.,SMI8300,"scanning, FIB",,,", 1990",UNSPEC,"SDI Fabsurplus LLC, Sumitomo Mitsui Finance an...",", ROHM Tsukuba Co.,Ltd",N,Y,Y
5,MO10004,Metrology,MA2052,Seiko Instrument Inc.,SPA500,", INSPECTION, AFM","SCANNING ATOMIC FORC, , ASSET No : 44000193-,",", #10200187000065",", 2004, 2003","UNSPEC, AVAIL",", Samsung Electronics Co., Ltd., ATREG, MITSUB...",", Samsung Electronics Co., Ltd., ATREG, Sumito...",N,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259703,MO9957,WET,MA2037,SHINKO SEIKI,SPR-30MLP2G,DRY,BUMP,12092006,,AVAIL,GEF,GEF,N,N,Y
259704,MO9960,Metrology,MA2039,Shinsung ENG,SGL-30,", QE",,"1110006, 11-10-005",", 2011","AVAIL, UNSPEC","GEF, SKhynix Semiconductor (China) Ltd","GEF, SKhynix Semiconductor (China) Ltd",N,Y,Y
259706,MO997,Stepper,MA321,Canon,FPA-1550M2,", STEPPER (FPA#10) (OM, STEPPER(FPA#2), g-Line...",", Stepper",", 411290",", 1989, 1994, 1993","UNSPEC, AVAIL, NOT_AVAIL","Magnachip Semiconductor, TLC Equipment, Inc., ...",", Magnachip Semiconductor, ROHM Tsukuba Co.,Lt...",N,Y,Y
259707,MO998,Stepper,MA321,Canon,FPA-1550M3-W,", PHOTO G-LINE STEPPER, Stepper, ｇ線ステッパ, STEPP...",", STEPPER Main Body/Fi, A/E, RETICLE SIZE：φ125ｍｍ",", 101380M3W, 003349M3, 906183, 101382, 008361M...",", 1991, 1989, 1990","UNSPEC, AVAIL, NOT_AVAIL",", LUNS, Kawasaki Steel, Renesas Electronics Co...",", Hangzhou E-Giant Semiconductor Tech Co., LTD...",N,Y,Y


In [13]:
# 각 컬럼에 대해 함수 적용
model_df["MODEL_NAME"] = model_df["MODEL_NAME"].apply(clean_unknown_to_empty)
model_df["MAKER_NAME"] = model_df["MAKER_NAME"].apply(clean_unknown_to_empty)
model_df["Category_Name"] = model_df["Category_Name"].apply(clean_unknown_to_empty)
model_df["Process_NM"] = model_df["Process_NM"].apply(clean_unknown_to_empty)
model_df["MODEL_NAME"] = model_df["MODEL_NAME"].apply(normalize_query)
model_df["MAKER_NAME"] = model_df["MAKER_NAME"].apply(normalize_query)
model_df["Category_Name"] = model_df["Category_Name"].apply(normalize_query)
model_df["Process_NM"] = model_df["Process_NM"].apply(normalize_query)

In [14]:
# 처리
model_df["Process_NM"] = model_df["Process_NM"].apply(
    lambda x: " ".join(
        dict.fromkeys(  # 중복 제거 + 순서 유지
            word.strip() 
            for part in x.split(",")             # 콤마로 분리
            for word in part.strip().split()     # 각 항목 안의 공백 단위 분리
        )
    ) if isinstance(x, str) else x
)

In [15]:

class HybridRetriever:
    def __init__(self, uri, collection_name="hybrid", dense_embedding_function=None):
        self.uri = uri
        self.collection_name = collection_name
        self.embedding_function = dense_embedding_function
        self.use_reranker = True
        self.use_sparse = True
        self.client = MilvusClient(uri=uri)

    def build_collection(self):
        if isinstance(self.embedding_function.dim, dict):
            dense_dim = self.embedding_function.dim["dense"]
        else:
            dense_dim = self.embedding_function.dim

        tokenizer_params = {
            "tokenizer": "standard",
            "filter": [
                "lowercase",
                {
                    "type": "length",
                    "max": 200,
                },
                {"type": "stemmer", "language": "english"},
                {
                    "type": "stop",
                    "stop_words": [
                        "?",
                        "an",
                        "and",
                        "are",
                        "as",
                        "at",
                        "be",
                        "but",
                        "by",
                        "for",
                        "if",
                        "in",
                        "into",
                        "is",
                        "it",
                        "no",
                        "not",
                        "of",
                        "on",
                        "or",
                        "such",
                        "that",
                        "the",
                        "their",
                        "then",
                        "there",
                        "these",
                        "they",
                        "this",
                        "to",
                        "was",
                        "will",
                        "with",
                    ],
                },
            ],
        }

        schema = MilvusClient.create_schema()
        schema.add_field(
            field_name="pk",
            datatype=DataType.VARCHAR,
            is_primary=True,
            auto_id=True,
            max_length=100,
        )
        schema.add_field(
            field_name="content",
            datatype=DataType.VARCHAR,
            max_length=65535,
            analyzer_params=tokenizer_params,
            enable_match=True,
            enable_analyzer=True,
        )
        schema.add_field(
            field_name="sparse_vector", datatype=DataType.SPARSE_FLOAT_VECTOR
        )
        schema.add_field(
            field_name="dense_vector", datatype=DataType.FLOAT_VECTOR, dim=dense_dim
        )
        # schema.add_field(
        #     field_name="original_uuid", datatype=DataType.VARCHAR, max_length=128
        # )
        # schema.add_field(field_name="doc_id", datatype=DataType.VARCHAR, max_length=64)
        # schema.add_field(
        #     field_name="chunk_id", datatype=DataType.VARCHAR, max_length=64
        # ),
        schema.add_field(field_name="original_index", datatype=DataType.VARCHAR, max_length=100,)

        functions = Function(
            name="bm25",
            function_type=FunctionType.BM25,
            input_field_names=["content"],
            output_field_names="sparse_vector",
        )

        schema.add_function(functions)

        index_params = MilvusClient.prepare_index_params()
        index_params.add_index(
            field_name="sparse_vector",
            index_type="SPARSE_INVERTED_INDEX",
            metric_type="BM25",
        )
        index_params.add_index(
            field_name="dense_vector", index_type="FLAT", metric_type="IP"
        )

        self.client.create_collection(
            collection_name=self.collection_name,
            schema=schema,
            index_params=index_params,
        )

    def insert_data(self, chunk, metadata):
        embedding = self.embedding_function([chunk])
        if isinstance(embedding, dict) and "dense" in embedding:
            dense_vec = embedding["dense"][0]
        else:
            dense_vec = embedding[0]
        self.client.insert(
            self.collection_name, {"dense_vector": dense_vec, **metadata}
        )

    def insert_data_batch(self, chunks, metadata_list):
        """
        chunks: List[str] - 임베딩할 텍스트 리스트
        metadata_list: List[Dict] - 각 텍스트에 대응하는 메타데이터 리스트
        """
        assert len(chunks) == len(metadata_list), "chunks와 metadata_list 길이가 다릅니다."

        # 배치 임베딩
        embeddings = self.embedding_function(chunks)

        # dense vector 추출
        if isinstance(embeddings, dict) and "dense" in embeddings:
            dense_vecs = embeddings["dense"]
        else:
            dense_vecs = embeddings  # 이미 dense만 있을 경우

        # Milvus insert용 리스트 생성
        entities = []
        for dense_vec, metadata in zip(dense_vecs, metadata_list):
            entity = {"dense_vector": dense_vec}
            entity.update(metadata)
            entities.append(entity)

        # insert
        self.client.insert(self.collection_name, entities)


    def search(self, query: str, k: int = 20, mode="hybrid"):

        output_fields = [
            "content",
            "original_index",
        ]
        if mode in ["dense", "hybrid"]:
            embedding = self.embedding_function([query])
            if isinstance(embedding, dict) and "dense" in embedding:
                dense_vec = embedding["dense"][0]
            else:
                dense_vec = embedding[0]

        if mode == "sparse":
            results = self.client.search(
                collection_name=self.collection_name,
                data=[query],
                anns_field="sparse_vector",
                limit=k,
                output_fields=output_fields,
            )
        elif mode == "dense":
            results = self.client.search(
                collection_name=self.collection_name,
                data=[dense_vec],
                anns_field="dense_vector",
                limit=k,
                output_fields=output_fields,
            )
        elif mode == "hybrid":
            full_text_search_params = {"metric_type": "BM25"}
            full_text_search_req = AnnSearchRequest(
                [query], "sparse_vector", full_text_search_params, limit=k
            )

            dense_search_params = {"metric_type": "IP"}
            dense_req = AnnSearchRequest(
                [dense_vec], "dense_vector", dense_search_params, limit=k
            )

            results = self.client.hybrid_search(
                self.collection_name,
                [full_text_search_req, dense_req],
                ranker=RRFRanker(),
                limit=k,
                output_fields=output_fields,
            )
        else:
            raise ValueError("Invalid mode")
        return [
            {
                "original_index": doc["entity"]["original_index"],
                "content": doc["entity"]["content"],
                "score": doc["distance"],
            }
            for doc in results[0]
        ]


In [30]:
dense_ef = BGEM3EmbeddingFunction(use_fp16=False, device="cuda")
standard_retriever = HybridRetriever(
    uri="http://172.17.0.1:19530",
    collection_name="milvus_equipment",
    dense_embedding_function=dense_ef,
)


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 27013.55it/s]


In [ ]:
standard_retriever.client.drop_collection("milvus_equipment")
standard_retriever.build_collection()

MODEL_ID                0
Category_Name           0
MAKER_ID                0
MAKER_NAME              0
MODEL_NAME              0
Process_NM              0
Configuration           0
SERIAL_NO               0
VINTAGE                 0
Tracker_availability    0
Seller                  0
Owner                   0
P&C                     0
STD_MODEL_YN            0
STD_MAKER_YN  

In [25]:
from tqdm import tqdm

is_insert = True
batch_size = 1024

if is_insert:
    standard_retriever.build_collection()

    chunks = []
    metadata_list = []

    for i, row in tqdm(model_df.iterrows(), total=len(model_df)):
        content = f"model: {str(row['MODEL_NAME'])} maker: {str(row['MAKER_NAME'])} category: {str(row['Category_Name'])}".strip()
        chunks.append(content)
        metadata_list.append({
            "original_index": str(row["MODEL_ID"]),
            "content": content,
        })

        if len(chunks) >= batch_size:
            standard_retriever.insert_data_batch(chunks, metadata_list)
            chunks = []
            metadata_list = []

    # 마지막 잔여
    if chunks:
        standard_retriever.insert_data_batch(chunks, metadata_list)


pre tokenize: 100%|██████████| 64/64 [00:00<00:00, 4536.07it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 40/40 [00:00<00:00, 113.81it/s]


In [31]:
test_set_df = pd.read_csv("../../data/test_dataset.csv")
test_set_df=test_set_df.dropna()

In [32]:
test_set_df.head()

,modelID,category,modelAlias,makerAlias
0,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai
1,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai automation
2,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai co.
3,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai corporation
4,MO736784,RF,p50e 12 n,cosel


In [33]:
test_set_df=test_set_df.drop_duplicates(subset=["category", "modelAlias"])
test_set_df = test_set_df.copy()

In [34]:
test_set_df=test_set_df[test_set_df["category"]!="Part Others"]
test_set_df=test_set_df[test_set_df["category"]!="PCBs"]
test_set_df=test_set_df[test_set_df["category"]!="RF"]
test_set_df=test_set_df[test_set_df["category"]!="CMP Parts & Consumables"]
test_set_df=test_set_df[test_set_df["category"]!="Motion Control"]
test_set_df=test_set_df[test_set_df["category"]!="Robot"]
test_set_df=test_set_df[test_set_df["category"]!="Chuck & Pedestal"]
test_set_df=test_set_df[test_set_df["category"]!="Process Kit"]
test_set_df=test_set_df[test_set_df["category"]!="MFC & LFC"]
test_set_df=test_set_df[test_set_df["category"]!="Gauge"]
test_set_df=test_set_df[test_set_df["category"]!="Valve"]
test_set_df=test_set_df[test_set_df["category"]!="Chiller & Scrubber"]
test_set_df=test_set_df[test_set_df["category"]!="Pump"]
test_set_df=test_set_df[test_set_df["category"]!="Laser"]

In [35]:
test_set_df.iloc[2100:2120]

,modelID,category,modelAlias,makerAlias
552815,MO2533,Etch,lam 9600 classic s/w,amat
552895,MO2533,Etch,lam tcp9600,amat
552975,MO2533,Etch,lam9600,amat
553055,MO2533,Etch,r9600,amat
553135,MO2533,Etch,rainbow 9600,amat
553215,MO2533,Etch,research 9600,amat
553295,MO2533,Etch,stand alone 9600 metal etch,amat
553375,MO2533,Etch,tc9600,amat
553455,MO2533,Etch,tcp 9600,amat
553535,MO2533,Etch,tcp 9600 dsq rf match,amat


In [36]:
test_set_df["category"] = test_set_df["category"].apply(normalize_query)
test_set_df["modelAlias"] = test_set_df["modelAlias"].apply(normalize_query)
test_set_df["makerAlias"] = test_set_df["makerAlias"].apply(normalize_query)

In [37]:
from tqdm import tqdm

slice_size   = 1000      # 구간 폭
step         = 5000     # 1만 단위 점프
max_start    = 43000    # 마지막 시작 인덱스
for search_name in ["hybrid", "dense", "sparse"]:
    for start in range(0, max_start + 1, step):        # 0, 10000, 20000, ...
        end   = start + slice_size                     # 1000개씩
        
        subset = test_set_df.iloc[start:end]
        total  = len(subset)
        correct_count = 0

        for idx, row in tqdm(test_set_df.iloc[start:end].iterrows(), total=total):
            model_id = row["modelID"]
            query = f"model: {str(row['modelAlias'])} maker: {str(row['makerAlias'])} category: {str(row['category'])}".strip()

            # 검색
            results = standard_retriever.search(query, mode=search_name, k=5)
            top_k_ids = [r["original_index"] for r in results]

            # 정답 포함 여부 확인
            if model_id in top_k_ids:
                correct_count += 1
                #print(f"정답: {model_id}, 쿼리: {query}, 검색결과: {top_k_ids}")
            # else:
            #     print(f"정답: {model_id}, 쿼리: {query}, 검색결과: {top_k_ids}")
            

        accuracy = correct_count / total
        print(start, end)
        print(f"정확도: {accuracy:.4f} ({correct_count}/{total})")


  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: No CUDA GPUs are available

In [ ]:
correct_count = 0
for idx, row in tqdm(test_set_df.iloc[2000:2100].iterrows(), total=100):
    model_id = row["modelID"]
    query = f"model: {str(row['modelAlias'])} maker: {str(row['makerAlias'])} category: {str(row['category'])}".strip()

    # 검색
    results = standard_retriever.search(query, mode=search_name, k=5)
    top_k_ids = [r["original_index"] for r in results]

    # 정답 포함 여부 확인
    if model_id in top_k_ids:
        correct_count += 1
        #print(f"정답: {model_id}, 쿼리: {query}, 검색결과: {top_k_ids}")
    # else:
    #     print(f"정답: {model_id}, 쿼리: {query}, 검색결과: {top_k_ids}")
    

accuracy = correct_count / total
print(start, end)
print(f"정확도: {accuracy:.4f} ({correct_count}/{total})")


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:07<00:00, 14.02it/s]

21000 22000
정확도: 0.0560 (56/1000)


In [ ]:
input_name="" 
input_maker=""
input_category="ate etc"
results = standard_retriever.search(f"model: {str(input_name)} maker: {str(input_maker)} category: {str()}", mode="dense", k=5)
print(results)


[{'original_index': 'MO84951', 'content': 'model:  maker: amat category: fab others', 'score': 0.8138582706451416}, {'original_index': 'MO7596', 'content': 'model:  maker: amat category: fab others', 'score': 0.813858151435852}, {'original_index': 'MO4591', 'content': 'model: fi maker: amat category: cmp', 'score': 0.8054901361465454}, {'original_index': 'MO943462', 'content': 'model:  maker: amat category: utility & fitting', 'score': 0.8045918941497803}, {'original_index': 'MO943265', 'content': 'model:  maker: amat category: machining parts', 'score': 0.800843358039856}]


In [161]:
# 권장 방법
test_set_df.loc[test_set_df["modelID"] == "MO112", "makerAlias"] = "lam research"


In [26]:
test_set_df.iloc[20000:21000]

,modelID,category,modelAlias,makerAlias
30474943,MO272711,ate etc,icos ci t130,icos
30474953,MO272711,ate etc,icos ci t130 ic inspection,icos
30474963,MO272711,ate etc,t130,icos
30474973,MO341591,asher,ecolite xp,피에스케이
30474996,MO250805,asher,ecolite2000,피에스케이
...,...,...,...,...
30523656,MO28189,microscope,smz 2t,(株)ﾆｺﾝ
30523706,MO28189,microscope,smz 2t (&#23455;&#20307;&#38997;微鏡),(株)ﾆｺﾝ
30523756,MO28189,microscope,smz 2t (microscope),(株)ﾆｺﾝ
30523806,MO297015,microscope,smz650,(株)ﾆｺﾝ


In [45]:
model_df[model_df["MODEL_ID"]=="MO1845"]

,MODEL_ID,Category_Name,MAKER_ID,MAKER_NAME,MODEL_NAME,Process_NM,Configuration,SERIAL_NO,VINTAGE,Tracker_availability,Seller,Owner,P&C,STD_MODEL_YN,STD_MAKER_YN
4905,MO1845,cvd,MA110,amat,p5000,teos/nitride p cvd sin sin_sion sio pe (tf)bps...,"MarkII, 2x DLH, 2x E, Asset No. : 40000292, ch...",", 4553, 309807, 4211, 11802, 799, E6237, 4590,...",", 1989, 2009, 1993, 1997, 2004, 2006, 1991, 20...","UNSPEC, RUNNING, AVAIL, NOT_AVAIL",", HANSON DING PACIFIC, INC, Hystars Semiconduc...",", Hystars Semiconductor (Wuxi) Co., Ltd., LG I...",", N, Y",Y,Y


In [ ]:
from tqdm import tqdm

slice_size   = 1000      # 구간 폭
step         = 10000     # 1만 단위 점프
max_start    = 200000    # 마지막 시작 인덱스

for search_name in ["hybrid", "dense", "sparse"]:
    print(f"\n=== {search_name.upper()} ===")
    for start in range(0, max_start + 1, step):        # 0, 10000, 20000, ...
        end   = start + slice_size                     # 1000개씩
        subset = model_df.iloc[start:end]
        total  = len(subset)

        correct_count = 0
        for idx, row in tqdm(subset.iterrows(), total=total, desc=f"{start}:{end}"):
            model_id = row["modelID"]
            query = (
                f"model: {str(row['modelAlias'])} "
                f"maker: {str(row['makerAlias'])} "
                f"category: {str(row['category'])}"
            ).strip()

            results   = standard_retriever.search(query, mode=search_name, k=1)
            top_k_ids = [r["original_index"] for r in results]

            if model_id in top_k_ids:
                correct_count += 1

        accuracy = correct_count / total
        print(f"[{start}:{end}]  정확도 = {accuracy:.4f} ({correct_count}/{total})")


In [23]:
test_set_df.iloc[60000:61010]

,modelID,category,modelAlias,makerAlias
4275543,MO712663,part others,ts210 309191 1,東京エレクトロン
4275615,MO712672,part others,ts210 309202 2,東京エレクトロン
4275687,MO712674,part others,ts210 309204 1,東京エレクトロン
4275759,MO712685,part others,ts210 309219 1,東京エレクトロン
4275831,MO712688,part others,ts210 309222 1,東京エレクトロン
...,...,...,...,...
4347903,MO712561,part others,ts210 309033 3,東京エレクトロン
4347975,MO712567,part others,ts210 309043 1,東京エレクトロン
4348047,MO712609,part others,ts210 309106 1,東京エレクトロン
4348119,MO712613,part others,ts210 309110 1,東京エレクトロン


In [21]:
model_df[model_df["MODEL_ID"]=="MO691221"]

,MODEL_ID,Category_Name,MAKER_ID,MAKER_NAME,MODEL_NAME,Process_NM,Configuration,SERIAL_NO,VINTAGE,Tracker_availability,Seller,Owner,P&C,STD_MODEL_YN,STD_MAKER_YN


In [14]:
input_name="ts1281 001112 13" 
input_maker="東京エレクトロン"
categoris="part others"
results = standard_retriever.search(f"model: {str(input_name)} maker: {str(input_maker)} category: {str(categoris)}", mode="hybrid", k=20)
print(results)


[{'original_index': 'MO364879', 'content': 'model: ts3r81 000019 12 maker: tokyo electron america inc. category: part others', 'score': 0.016393441706895828}, {'original_index': 'MO767278', 'content': 'model: ts3281 001020 13, 3280 006301 13 maker: tel category: part others', 'score': 0.016393441706895828}, {'original_index': 'MO364881', 'content': 'model: ts3r81 000037 12 maker: tokyo electron america inc. category: part others', 'score': 0.016129031777381897}, {'original_index': 'MO753158', 'content': 'model: 13.56mhz_13.56mhz/30db maker: seowoo electron category: part others', 'score': 0.016129031777381897}, {'original_index': 'MO364810', 'content': 'model: ts3281 000080 12+ maker: tokyo electron america inc. category: part others', 'score': 0.01587301678955555}, {'original_index': 'MO369790', 'content': 'model: fu 13 maker:  category: part others', 'score': 0.01587301678955555}, {'original_index': 'MO365689', 'content': 'model: ts3385 000865 13 maker: tokyo electron america inc. ca

# 검증

In [ ]:
def load_jsonl(file_path: str):
    """Load JSONL file and return a list of dictionaries."""
    with open(file_path, "r") as file:
        return [json.loads(line) for line in file]


dataset = load_jsonl("evaluation_set.jsonl")
k = 5

mode = "hybrid"

total_query_score = 0
num_queries = 0

for query_item in dataset:

    query = query_item["query"]

    golden_chunk_uuids = query_item["golden_chunk_uuids"]

    chunks_found = 0
    golden_contents = []
    for doc_uuid, chunk_index in golden_chunk_uuids:
        golden_doc = next(
            (doc for doc in query_item["golden_documents"] if doc["uuid"] == doc_uuid),
            None,
        )
        if golden_doc:
            golden_chunk = next(
                (
                    chunk
                    for chunk in golden_doc["chunks"]
                    if chunk["index"] == chunk_index
                ),
                None,
            )
            if golden_chunk:
                golden_contents.append(golden_chunk["content"].strip())

    results = standard_retriever.search(query, mode=mode, k=5)

    for golden_content in golden_contents:
        for doc in results[:k]:
            retrieved_content = doc["content"].strip()
            if retrieved_content == golden_content:
                chunks_found += 1
                break

    query_score = chunks_found / len(golden_contents)

    total_query_score += query_score
    num_queries += 1
